In [1]:
import os 
import pandas as pd

In [ ]:
# load d-genies association table for chromosome assignment based on homology to Pst104E
df = pd.read_csv("psh_nepal.hic.hap12.p_ctg_Pst104E-1_assoc.tsv", sep="\t")
# load blast results
blast = pd.read_csv("all.blast", sep="\t", header=None, names="qseqid sseqid stitle length qcovs pident qstart qend sstart send sscinames staxids evalue".split(" "))
blast["pident"] = blast["pident"].astype(float)
# load windowed coverage
cov = pd.read_csv("coverage/window-stats.tsv", sep="\t", usecols=(0,2))

In [52]:
blast

,qseqid,sseqid,stitle,length,qcovs,pident,qstart,qend,sstart,send,sscinames,staxids,evalue
0,h1tg000001l_0,gi|2234918871|ref|XM_047955534.1|,Puccinia striiformis f. sp. tritici uncharacte...,4069,1,100.000,241613,245681,7679,3611,Puccinia striiformis f. sp. tritici,168172,0.000000e+00
1,h1tg000001l_0,gi|2234918871|ref|XM_047955534.1|,Puccinia striiformis f. sp. tritici uncharacte...,2799,1,99.964,237259,240057,11748,8950,Puccinia striiformis f. sp. tritici,168172,0.000000e+00
2,h1tg000001l_0,gi|2234918871|ref|XM_047955534.1|,Puccinia striiformis f. sp. tritici uncharacte...,2190,1,99.954,247054,249243,2642,453,Puccinia striiformis f. sp. tritici,168172,0.000000e+00
3,h1tg000001l_0,gi|2234918871|ref|XM_047955534.1|,Puccinia striiformis f. sp. tritici uncharacte...,894,1,99.888,240152,241045,8949,8056,Puccinia striiformis f. sp. tritici,168172,0.000000e+00
4,h1tg000001l_0,gi|2234918871|ref|XM_047955534.1|,Puccinia striiformis f. sp. tritici uncharacte...,507,1,100.000,236486,236992,12362,11856,Puccinia striiformis f. sp. tritici,168172,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106538,h2tg000110l,gi|2413979308|gb|CP110423.1|,Puccinia triticina strain Pt15 chromosome 3A,426,6,83.099,22413,22826,6796602,6796196,Puccinia triticina,208348,3.100000e-92
106539,h2tg000110l,gi|2413979308|gb|CP110423.1|,Puccinia triticina strain Pt15 chromosome 3A,119,6,95.798,22155,22273,6798119,6798003,Puccinia triticina,208348,1.200000e-41
106540,h2tg000110l,gi|2413979308|gb|CP110423.1|,Puccinia triticina strain Pt15 chromosome 3A,105,6,84.762,23131,23231,6795943,6795840,Puccinia triticina,208348,2.080000e-14
106541,h2tg000110l,gi|334361375|gb|HM147441.1|,Puccinia striiformis voucher DAOM 240067 cytoc...,371,1,98.922,17103,17473,371,1,Puccinia striiformis,27350,0.000000e+00


In [72]:
# get mtDNA contigs with pident with mtDNA reference >= 80
mtDNA = blast[(blast["stitle"].str.contains("mitochondr", case=False)) & (blast["pident"]>=80)]
mtDNA_contigs = list(mtDNA["qseqid"].unique())

# check if there are contigs longer than 1Mbp with mtDNA hits (whose name contains "_" as they are split into chunks)
# these can be from NUMTs and will not be identified as mtDNA contig subsequently
for n in mtDNA_contigs:
    if "_" in n:
        print(n)

h1tg000012l_5
h1tg000015l_0
h1tg000019l_4
h2tg000011l_4
h2tg000017l_3


In [54]:
# get mean window coverage per contig
cov.columns = ["Query", "mean_cov"]
cov["Query"] = cov["Query"].apply(lambda x: x.split(":")[0])
cov["mean_cov"] = cov["mean_cov"].astype(float)
mean_cov_per_ctg = cov.groupby("Query")["mean_cov"].mean().reset_index()
mean_cov_per_ctg

,Query,mean_cov
0,h1tg000001l,46.698550
1,h1tg000002l,48.978347
2,h1tg000003l,48.449069
3,h1tg000004l,80.062176
4,h1tg000005l,48.358627
...,...,...
570,h2tg000106l,102.109643
571,h2tg000107l,28.979941
572,h2tg000108l,77.624064
573,h2tg000109l,192.374874


In [ ]:
df2 = pd.merge(df, mean_cov_per_ctg, on="Query")
df2["has_mtDNA_hit"] = df2["Query"].isin(mtDNA_contigs)
df2

,Query,Target,Strand,Q-len,Q-start,Q-stop,T-len,T-start,T-stop,mean_cov,has_mtDNA_hit
0,h1tg000033l,chr1,+,10228,7047,9924,5601519,1438413,1438470,46.647622,True
1,h1tg000137l,chr1,+,8885,2095,2152,5601519,1438413,1438470,54.380979,True
2,h1tg000139l,chr1,-,4668,920,977,5601519,1438413,1438470,9.726435,True
3,h1tg000172l,chr1,+,4203,2499,2556,5601519,1438413,1438470,16.400428,True
4,h1tg000302l,chr1,-,6285,464,521,5601519,1438413,1438470,19.418775,True
...,...,...,...,...,...,...,...,...,...,...,...
570,h1tg000447l,NaN,+,9670,na,na,na,na,na,16.511789,True
571,h1tg000451l,NaN,+,3961,na,na,na,na,na,15.529159,False
572,h2tg000044l,NaN,+,6126,na,na,na,na,na,8.375612,True
573,h2tg000091l,NaN,+,11329,na,na,na,na,na,22.453403,False


In [71]:
filt = df2[(df2["mean_cov"]>=10)&(df2["Q-len"]>=20000)&(df2["has_mtDNA_hit"]==False)]
filt

,Query,Target,Strand,Q-len,Q-start,Q-stop,T-len,T-start,T-stop,mean_cov,has_mtDNA_hit
12,h1tg000002l,chr1,+,5377875,1011,5377134,5601519,3184,5598249,48.978347,False
13,h2tg000001l,chr1,+,5405112,451,5404755,5601519,4100,5599704,48.304371,False
14,h1tg000012l,chr2,-,5389257,90309,5385183,5438820,1099,5404098,49.325045,False
15,h2tg000004l,chr2,-,5598330,63601,5596317,5438820,1086,5404098,48.680548,False
68,h2tg000003l,chr3,-,5353409,372,5345288,5383601,8495,5380796,47.697853,False
...,...,...,...,...,...,...,...,...,...,...,...
500,h2tg000009l,chr16,+,3063723,14082,3040924,3164667,2162,3114714,47.968409,False
501,h1tg000007l,chr17,+,2591081,4641,2590322,2843575,944,2828860,48.741996,False
502,h2tg000007l,chr17,+,2603707,3990,2603316,2843575,944,2829896,48.552408,False
504,h1tg000001l,chr18,-,2525688,1119,2525185,2627259,9431,2608670,46.698550,False


In [ ]:
df2.to_csv("coverage/psh_nepal.hic.hap12.p_ctg_Pst10W4E-1_assoc_raw.tsv", sep="\t", index=None)
filt.to_csv("coverage/psh_nepal.hic.hap12.p_ctg_Pst104E-1_assoc_filtered.tsv", sep="\t", index=None)

seqtk subseq ../psh_nepal.hic.hap12.p_ctg.fasta psh_nepal.hic.hap12.p_ctg_Pst104E-1_assoc_filtered.contig_names.txt > psh_nepal.v2.filtered_contigs.fasta